# Importing Data from Youtube using API Calls

- I want to extract data like Video ID , Video Title, and each Videos statictics like no of comments , likes, views.
- I Created an API key from the Google Cloud Youtuble Video Data API for me to Extract this data using API
- I Took StatQuest as my Channel to extract and got the Channel ID
- I then did API calls to extract all the data into a Data Frame for me to Store it.

### Importing Libraries  

In [1]:
import requests
import pandas as pd
import time

### Assigning the API Key and Channel ID
- Here I hid my API Key here 
- I took Channel Id of the Most subscribed channel in Youtube : T-Series

In [2]:
API_KEY = "xxxxxxx"
CHANNEL_ID = "UCq-Fj5jknLsUf-MWSy4_brA"

### Getting the Video Stats with Second API Call
- Here I commented the Dislike count as youtube no longer has dislike count and throws an error
- Here we get count of Views,likes and comments of each video in the channel with this call.

In [3]:
def get_video_stats(video_id):
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json() 
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    
    return video_id,video_title,upload_date,view_count,like_count,comment_count

### First API call and Storing in the Dataframe 
- The first API call is made here and the details like Video Id, Video title, When it is uploaded are extracted here
- The sleep function is used here to give API a time gap inorder for it to not immediately start the extraction process
- Page Token helps is extracting data from other pages as well.
- An "If statement" is added to extract only if its a youtube video and to ensure it does'nt grab anything else in the       data.
- I have added Error handling to make sure to only return when api calls are successfull and stop when it reaches the page   end.
- It finally returns the DataFrame with all the Details of the Videos in the Channel

In [4]:
def get_videos(df):  
    pageToken=""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken
        response = requests.get(url).json()
        time.sleep(5)
    
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]

                get_video_stats(video_id)
                #Saving all the Data into a DataFrame
                df = df.append({'video_id':video_id,'video_title':video_title,'upload_date':upload_date,
                                'view_count':view_count,'like_count':like_count,                         
                                'comment_count':comment_count},ignore_index=True)
        try:
            if response.status_code != 200:
                break
            elif response['nextPageToken'] != None: #if none,it reached the last page and it breaks out of the loop 
                pageToken = "pageToken=" + response['nextPageToken']
      
        except :
            break   
            
    return df

### Dataframe Creation
- Here we create the dataframe with the Column Titles of the information we are about to extract
- We,then pass the dataframe to the get_videos Function to start the API Calls

In [12]:
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"])

df = get_videos(df)

## View of the DataFrame 

In [13]:
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,ODUNnVFBZvw,Teaser: Mere Dil Gaaye Ja (Zooby Zooby)- Dhokh...,2022-08-29,1226429,34584,883
1,CFXSGDyi3Aw,"Jubin Nautiyal, Payal Dev: Meethi Meethi | Ras...",2022-08-29,1226429,34584,883
2,LCl2_jiPtpg,"Sonu Nigam, Anuradha Paudwal: Woh Ho Tum (HD F...",2022-08-28,1226429,34584,883
3,1-YmsbITsx8,Anand Raj Anand: Oye Raju Pyar Na Kariyo | Gov...,2022-08-28,1226429,34584,883
4,A9mdcw2nk48,"Tu Pyar Hai Kisi Aur Ka | Kumar Sanu, Anuradha...",2022-08-28,1226429,34584,883
5,J6CyJY76KG4,#LoveStatus ❤️#lovesong 💘 Woh Ho Tum 👩‍❤️‍👩 #S...,2022-08-28,1226429,34584,883
6,Zpr02dkVSYc,#SadSongStatus 💔Oye Raju Pyar Na Kariyo 💔#Govi...,2022-08-28,1226429,34584,883
7,xav1vPVeEts,#KumarSanu | Tu Pyaar Hai Kisi Aur Ka 💔 #SadSt...,2022-08-28,1226429,34584,883
8,JeJXJd8dUQ4,Jubin Nautiyal: Teri Galliyon Se (Lyrical) | G...,2022-08-28,1226429,34584,883
9,rY_0SOlmrdY,"Ankhiyon Se Goli Maare | Sonu Nigam, Jaspinder...",2022-08-28,1226429,34584,883
